# Group Interest Points

### Prepare packages

In [1]:
import s3fs
fs = s3fs.S3FileSystem(anon=False)

In [2]:
import boto3
s3 = boto3.client("s3")

In [9]:
import pandas as pd
import pyarrow.parquet as pq

### Input data

In [54]:
table_from_parquet = pq.ParquetDataset("s3://ecb-hackathon-202006-dbn-group1/data/OSM/InterestPointsFinal",filesystem=fs).read().to_pandas()

In [71]:
financials_from_parquet = pq.ParquetDataset("s3://ecb-hackathon-202006-dbn-group1/data/OSM/InterestPointsFinalFinance.parquet",filesystem=fs).read().to_pandas()

We decided to separate shops that are particularly big (malls and such) and those that sell very expensive items, are they are less linked to cash spending. <br>
We also merge amenities and leisures, as we don't want to overcomplicate our model

In [29]:
big_shops = ['department_store', 'general', 'mall', 'supermarket', 'wholesale', 'atv', 'car', 'boat', 'caravan', 'jetski', 'motorcycle', 'ski', 'snowmobiles', 'swimming_pool', 'trailer', 'tyres']

In [57]:
table_from_parquet.loc[table_from_parquet[table_from_parquet.VALUE.apply(lambda x: x in big_shops)].index,'KEY'] = 'big_shop'

In [58]:
table_from_parquet.loc[table_from_parquet[table_from_parquet.KEY=='leisure'].index,'KEY'] = 'amenity'

### Compute and save (wide)

In [60]:
IP_features = table_from_parquet.groupby(['KEY', 'NUTS_ID', 'NUTS_NAME']).COUNT.sum().reset_index()

In [61]:
IP_features.to_csv("s3://ecb-hackathon-202006-dbn-group1/data/OSM/IP_features.csv")

In [69]:
IP_features_wide = IP_features.pivot(index = 'NUTS_ID', columns = 'KEY', values ='COUNT')

In [70]:
IP_features_wide.to_csv("s3://ecb-hackathon-202006-dbn-group1/data/OSM/IP_features_wide.csv")

In [73]:
financials = financials_from_parquet.groupby(['NUTS_ID','VALUE']).COUNT.sum()

In [77]:
financials_wide = financials.reset_index().pivot(index = 'NUTS_ID', columns = 'VALUE', values ='COUNT')

In [78]:
financials_wide.to_csv("s3://ecb-hackathon-202006-dbn-group1/data/OSM/financials_features_wide.csv")